In [2]:
import json
import pandas
import utils
import itertools
from itertools import combinations_with_replacement
import copy

# Get the character_stats, weapon stats and artifact stats that you want

In [3]:
character = "characters/soldier11.json"
weapon = "weapons/starlight_engine_r5.json"
disk = "disks_setup/disk_options.json"
# excel_save_name = "results/ellen_sig.xlsx"

# other constants
constants = "constants.json"
target_path = "enemies/tyrfing.json"

substat_points = 34

damage_combo = ["ultimate"]

# Real calculations begin

In [4]:
# load character, weapon, disk, and constants
character_stats, base_motion_values, weapon_stats, other_stats = utils.get_stats(character, weapon, disk)
disk_stats = utils.evaluate_disk_stat(disk_options = other_stats)
total_stats, target_stats = utils.get_total_stats(
    character_stats,
    weapon_stats,
    disk_stats,
    target_path
)
new_mv = utils.motion_to_damage(base_motion_values, total_stats, target_stats)
final_damage = utils.combo_to_damage(damage_combo, new_mv)
print(final_damage)

129239.3895631893


In [5]:
character_stats, base_motion_values, weapon_stats, other_stats = utils.get_stats(character, weapon, disk)

In [6]:
def disk_possibilities(
    character_stats,
    base_motion_values,
    weapon_stats,
    other_stats,
    substat_points = 2,
    disk_4_attributes_of_interest = {
        "ATK_%": 0.3,
        "CRIT_Rate": 0.24,
        "CRIT_DMG" : 0.48
    },
    disk_5_attributes_of_interest = {
        "ATK_%": 0.3,
        "PEN_Ratio": 0.24,
        "Elemental_Damage": 0.3
    },
    disk_6_attributes_of_interest = {
        "ATK_%": 0.3
    },
    sub_attributes_of_interest = {
        "Flat_ATK": 22,
        "ATK_%": 0.03,
        "CRIT_Rate": 0.024,
        "CRIT_DMG" : 0.048,
        "Flat_PEN": 9
    }
):
    best_damage = 0
    best_allocation = None
    for disk_4_allocation in combinations_with_replacement(
        disk_4_attributes_of_interest,
        1
    ):
        for disk_5_allocation in combinations_with_replacement(
            disk_5_attributes_of_interest,
            1
        ):
            for disk_6_allocation in combinations_with_replacement(
                disk_6_attributes_of_interest,
                1
            ):
                main_stat_dict = {
                    "disk_4": disk_4_allocation[0],
                    "disk_5": disk_5_allocation[0],
                    "disk_6": disk_6_allocation[0],
                }
                for sub_allocation in combinations_with_replacement(sub_attributes_of_interest, substat_points):
                    # make a copy of every dictionary in case we directly modify the dictionary
                    character_stats_copy = copy.deepcopy(character_stats)
                    base_motion_values_copy = copy.deepcopy(base_motion_values)
                    weapon_stats_copy = copy.deepcopy(weapon_stats)
                    other_stats_copy = copy.deepcopy(other_stats)
                    
                    # Count occurrences of each substat in the allocation
                    substat_allocations = {stat: sub_allocation.count(stat) for stat in sub_attributes_of_interest.keys()}
                    # make a copy of every dictionary in case we directly modify the dictionary
                    disk_stats = utils.evaluate_disk_stat(
                        disk_options = other_stats_copy,
                        main_stat_dict = main_stat_dict,
                        sub_stat_dict = substat_allocations
                    )
                    total_stats, target_stats = utils.get_total_stats(
                        character_stats_copy,
                        weapon_stats_copy,
                        disk_stats,
                        target_path
                    )
                    new_mv = utils.motion_to_damage(base_motion_values_copy, total_stats, target_stats)
                    final_damage = utils.combo_to_damage(damage_combo, new_mv)
                    if final_damage > best_damage:
                        best_damage = final_damage
                        best_main_stats = main_stat_dict
                        best_allocation = substat_allocations
                        best_total_stats = total_stats

    print(best_damage)
    print(best_main_stats)
    print(best_allocation)


disk_possibilities(
    character_stats,
    base_motion_values,
    weapon_stats,
    other_stats,
    substat_points = 34)

139059.11133679998
{'disk_4': 'ATK_%', 'disk_5': 'PEN_Ratio', 'disk_6': 'ATK_%'}
{'Flat_ATK': 0, 'ATK_%': 0, 'CRIT_Rate': 0, 'CRIT_DMG': 0, 'Flat_PEN': 34}
